# ECG Image to Time Series Conversion

**Name:** Yehoshua Benjamin Perez Condori  
**Student ID:** 2231349

This notebook converts an ECG image into a time-series representation using image processing techniques.

In [ ]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Load Image
Update `image_path` to point to your ECG image file.

In [ ]:
image_path = "ecgreading.png"  # place image in same folder as notebook
img = cv2.imread(image_path)
img_gray_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

## Image Loading and Preprocessing

In [ ]:
def IMG_Load_and_Preprocess(image_path):
    image = cv2.imread(image_path)
    processed_image = np.zeros_like(image)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if np.all(image[i, j] <= 230):
                processed_image[i, j] = image[i, j]
            else:
                processed_image[i, j] = [255, 255, 255]

    img_gray = cv2.cvtColor(processed_image, cv2.COLOR_BGR2GRAY)

    df = pd.DataFrame(img_gray)
    plt.figure(figsize=(8, 6))
    plt.imshow(df, cmap='gray')
    plt.colorbar()
    plt.title('Time Series Representation')
    plt.axis('off')
    plt.show()

    df.replace(255, 0, inplace=True)
    df.to_csv('image_data.csv', index=False)

    return img_gray

## Image Processing Functions

In [ ]:
def Gaussian_Blur(img):
    return cv2.GaussianBlur(img, (3, 3), 0)

def Canny_Edge_Detection(img):
    return cv2.Canny(img, 75, 200)

def Find_Contours(edges):
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def Draw_Contours(image, contours):
    return cv2.drawContours(image.copy(), contours, -1, (255, 0, 0), 2)

## Plot Processing Stages

In [ ]:
def plot_images(original, gray, blurred, edges, contours_img):
    titles = ['Original', 'Grayscale', 'Blurred', 'Canny Edges', 'Contours']
    images = [original, gray, blurred, edges, contours_img]

    plt.figure(figsize=(15, 4))
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(images[i], cmap='gray')
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

## Run Pipeline

In [ ]:
preprocessed = IMG_Load_and_Preprocess(image_path)
blurred = Gaussian_Blur(preprocessed)

_, thresh = cv2.threshold(blurred.astype(np.uint8), 127, 255, 0)
edges = Canny_Edge_Detection(thresh)
contours = Find_Contours(edges)
contour_img = Draw_Contours(img_gray_rgb, contours)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
closed = cv2.morphologyEx(contour_img, cv2.MORPH_CLOSE, kernel)

plot_images(img_gray_rgb, preprocessed, blurred, edges, closed)